# Формирование ссылок

In [ ]:
# Создаем словарь, в котором каждому направлению соответствет набор из 5 постраничных ссылок
# Текст ссылки включает в себя ключевые слова по поиску, например "Лингвистическое образование" или "Лингвитика" и пагинацию

links = []
num = ['0','1','2','3','4','5']
links = {'Философское%20образование': [],  'Политология': [], 'Психология': [], 'История': [], 'Юриспруденция': [], 'Журналистика': [],
         'Международные%20отношения': [], 'Регионоведение': [], 'Редактирование': [], 'Филология': [], 'Лингвистический': [], 'Культурология': [],
         'Религиоведение': [], 'Документооборот': [], 'Физическая%20культура': [], 'Реклама': []}

for i in num:
    for key in links:
        links[key].append("https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=" + key + "&search_field=name&search_field=description&excluded_text=&area=113&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&employment=full&education=higher&order_by=relevance&search_period=30&items_on_page=20&page=" + i )

print(links)

{'Философское%20образование': ['https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=Философское%20образование&search_field=name&search_field=description&excluded_text=&area=113&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&employment=full&education=higher&order_by=relevance&search_period=30&items_on_page=20&page=0', 'https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=Философское%20образование&search_field=name&search_field=description&excluded_text=&area=113&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&employment=full&education=higher&order_by=relevance&search_period=30&items_on_page=20&page=1', 'https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=Философское%20образование&search_field=name&search_field=description&excluded_text=&area=113&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&employment=full&education=higher&order_by=relevance&search_period=30&items_on_page=

In [ ]:
# Проверяем получившиеся ссылки по направлениям
for link in links['Религиоведение']:
  print(link)

# Парсинг по отдельным направлениям

1) Для парсинга необходимо менять название направления и соответственно названиефинального словаря

2) Парсит по 20 вакансий со страницы (если есть)

In [ ]:
import requests as req
from bs4 import BeautifulSoup

result = []
salary_list = []

try:
  #for speciality in links:
    for link in links["Юриспруденция"]:


      session = req.session()
      session.headers = {
              'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
              'accept-language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,pl-PL;q=0.6,pl;q=0.5,en-US;q=0.4'
      }

      res = session.get(link)
      res.raise_for_status()
      html = res.text

      soup = BeautifulSoup(html, 'html5lib')
      jobs = soup.find_all("a", class_ ="serp-item__title")


      # Парсим ключевые навыки
      for job in jobs:

        vacancies = job.text   # Спарсили название вакансии
        print(vacancies)

        job_link = job.get("href")
        session = req.session()
        session.headers = {
                  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                  'accept-language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,pl-PL;q=0.6,pl;q=0.5,en-US;q=0.4'
          }
        res_skills = session.get(job_link)
        html_skills = res_skills.text

        soup_skills = BeautifulSoup(html_skills, 'html.parser')
        skills = soup_skills.find('div', class_='bloko-tag-list')

        if skills == None:
          key_skills = 'Навыки не указаны'
        else:
          skills = soup_skills.find('div', class_='bloko-tag-list')
          list_of_skills = []
          for skill in skills:
                list_of_skills.append(skill.text)
                key_skills = list_of_skills

        inner_salary = soup_skills.find("span", class_="bloko-header-section-2 bloko-header-section-2_lite")
        salary = ''.join(inner_salary.text.strip('руб.').strip('от').strip('до').split()[:2])
        digits = '1234567890'

        for i in salary:
          if i in digits:
            salary_int = int(salary)
          else:
            salary_int = 0
        print(salary_int)

        result.append({'Направление': 'Юриспруденция',
                'Вакансия': vacancies,
                'Доход': salary_int,
                'Ключевые навыки': key_skills
            })

except Exception as ex:
    print(f'Error: {ex}')

print(result)

In [ ]:
print(result)

In [ ]:
len(result)

# Импорт в CSV

In [ ]:
import csv

with open('Law.csv', 'w') as f:
  writer = csv.DictWriter(f, result[0].keys())
  writer.writeheader()
  for r in result:
    writer.writerow(r)

In [ ]:
# Открываем csv через датафрейм
import pandas as pd
dataset = pd.read_csv('law.csv')
dataset